In [22]:
import urllib
import urllib.request
import os
import time
import numpy as np
import os
from itertools import product
import base64

import urllib.request

def get_page(url):
    
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 

    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read().decode("iso-8859-1")
    #return urllib.request.urlopen(url).read().decode("iso-8859-1")
    return data

def is_int(txt):
    try:
        int(txt)
        return True
    except:
        return False


chlist = list(range(1,18218))

In [24]:
import numpy as np
from itertools import product

url=''
while(True):
    try:
        os.makedirs('../killersudoku-dailykillersudoku-collection', exist_ok=True)
        completion_dict = {}
        for _,_, files in os.walk('../killersudoku-dailykillersudoku-collection'):
            for file in files:
                completion_dict[int(file.split('.')[0][-5:])] = True
        for i in open('dailysudoku-brokens.txt').read().split():
            completion_dict[int(i)] = True

        for pzidx in chlist:
            if pzidx in completion_dict:
                continue

            url = 'https://www.dailykillersudoku.com/puzzle/%d'%(pzidx)
            filename = '../killersudoku-dailykillersudoku-collection/puzzle%05d.txt'%(pzidx)
            str_in = get_page(url)
                
            if 'Oops, no puzzles!' in str_in:
                open(filename, 'w').write("#No puzzle for url: "+url)
                continue
                
            str_in1 = str_in.split('"solution_base64":"')[1].split('"')[0]
            str_in2 = str_in.split('"board_base64":"')[1].split('"')[0]

            ans = np.array([i for i in base64.b64decode(str_in1)][2:])
            cellarr = np.array([i for i in base64.b64decode(str_in2)][3:][::2][:81]).reshape(9,9)

            cells = {}
            for i,j in product(range(9), range(9)):
                if cellarr[i,j] not in cells:
                    sm = np.sum(ans.flatten()[cellarr.flatten()==cellarr[i,j]])
                    line = []
                    for ii,jj in product(range(9), range(9)):
                        if cellarr[i,j] == cellarr[ii,jj]:
                            line.append((ii,jj))
                    cells[sm,line[0]] = line
                    
            txt = '#source: '+url+'\n'
            txt += '#reported difficulty: '+str_in.split('"difficulty":')[1].split(',')[0]+'\n'
            txt += '#creation_date: '+str_in.split('"date":"')[1].split('"')[0]+'\n\n'
            txt += '+'+('---+'*9)+'\n'
            for i in range(0,9):
                txt += '|'
                markers = []
                for j in range(0,9):
                    isin = False
                    for (sm, fval), vals in cells.items():
                        if (i,j) in vals:
                            break

                    txt += (str(sm)+'   ')[:3] if fval == (i,j) else '   '
                    isthick = (j+1)%3==0 and j<8 
                    txt += ('|║')[isthick] if (i,j+1) not in vals else '  '[isthick]
                    markers.append(not (i+1,j) in vals)
                txt += '\n'
                isthick = (i+1)%3==0 and i<8
                txt += '+' + '+'.join(('---','===')[isthick] if i else ('   ','   ')[isthick] for i in markers) + '+\n'

            os.makedirs(os.path.dirname(filename), exist_ok=True)
            open(filename, 'w').write(txt)
        break
    except Exception as e:
        if repr(e) == "Error('Incorrect padding',)":
            open(filename, 'w').write("#No puzzle for url: "+url)
        else:
            print('timeout...', url, '\n', repr(repr(e)))
            time.sleep(5)

In [21]:
with open('dailysudoku-brokens.txt', 'a') as f:
    for dir, _, files in os.walk('../killersudoku-dailykillersudoku-collection'):
        files.sort()
        for file in files:
            if 'puzzle for url' in open(dir+'/'+file).read():
                f.write(file.split('.')[0][-5:]+'\n')
                os.remove(dir+'/'+file)